In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
import pandascharm as pc

from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62
from Bio.SubsMat.MatrixInfo import blosum45
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align import AlignInfo

from sklearn.preprocessing import scale 
from sklearn import model_selection

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso


from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate
import scipy.stats as stats


C:\Users\memre\anaconda3\lib\site-packages\Bio\SubsMat\__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
def removeoutlier_col(df,cols):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1
    df_out = df[~((df[[cols]] < (Q1 - 1.5 * IQR)) |(df[[cols]] > (Q3 + 1.5 * IQR))).any(axis=1)]
    return df_out

In [3]:
blosum62.update(((b,a),val) for (a,b),val in list(blosum62.items()))
blosum45.update(((b,a),val) for (a,b),val in list(blosum45.items()))

def score_pairwise(seq1, seq2, matrix, gap_s, gap_e, gap = True):
    for A,B in zip(seq1, seq2):
        diag = ('-'==A) or ('-'==B)
        yield (gap_e if gap else gap_s) if diag else matrix[(A,B)]
        gap = diag


In [4]:
def encode(encoding, output, df_clean, aln, esm1b, key = None):
    

    ClustalAlign = AlignIO.read(aln, 'clustal')
    summary_align = AlignInfo.SummaryInfo(ClustalAlign )
    dframe = pc.from_bioalignment(ClustalAlign).transpose()
    sequences = dframe.loc[df_clean.index]
    
    y = df_clean['y_val']

    scaler = preprocessing.StandardScaler()
    
    
    if encoding == 'One-Hot-Encoder':

        one_hot = OneHotEncoder()
        encoded = one_hot.fit(sequences)
        X = encoded.transform(sequences).toarray()
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
           
    if encoding == 'Bag-of-Words':

        X = pd.DataFrame([ProteinAnalysis(i).count_amino_acids() for i in df_clean['Sequence']])
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
    if encoding == 'bigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                st = i+j
                all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-1):
                try:
                    check = temp[k] + temp[k+1]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
    
    if encoding == 'trigram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    st = i+j+k
                    all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-2):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
        
    if encoding == 'quadrogram':
        
        X = df_clean['Sequence']

        example = df_clean['Sequence'][0]
        lst = ['E','G','L','Y','T','H','R','A','C','D','P','I','F','N','K','S','V','M','W','Q']
        all_dct = {}
        key = []
        for i in lst:
            for j in lst:
                for k in lst:
                    for l in lst:
                        st = i+j+k+l
                        all_dct[st] = []

        for example, id in zip(X,range(len(X))):

            temp = list(example)
            temp_dct = dict.fromkeys(all_dct.keys(),0)
            for k in range(len(temp)-3):
                try:
                    check = temp[k] + temp[k+1]+temp[k+2]+temp[k+3]
                    temp_dct[check] += 1
                except:
                    pass
            for key, value in temp_dct.items():
                all_dct[key].append(value)
                
        X = pd.DataFrame.from_dict(all_dct).set_index(df_clean.index)
        X = np.array(X)
        scaler.fit(X)
        X_scaled = scaler.transform(X)

    if encoding == 'BLOSUM62':

        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum62, -5, -1))
                j += 1
            i += 1
        
        X = np.array(enc_seq)
        scaler.fit(enc_seq)
        X_scaled = scaler.transform(enc_seq)
        
        
        
    if encoding == 'BLOSUM45':
        
        n = len(sequences)
        enc_seq = np.zeros((n,n))

        i = 0

        for a in list(sequences.index):
            j = 0
            for b in list(sequences.index):
                enc_seq[i,j] = sum(score_pairwise(sequences.loc[a], sequences.loc[b], blosum45, -5, -1))
                j += 1
            i += 1
        
        X = np.array(enc_seq)   
        scaler.fit(enc_seq)
        X_scaled = scaler.transform(enc_seq)
        
    if encoding == 'ESM1b':
        

        encoded = esm1b.loc[df_clean.index]
        X = np.array(encoded)
        scaler.fit(X)
        X_scaled = scaler.transform(X)
        
    return X, y, X_scaled, scaler


In [5]:
def ml_process(encoding, output, df, aln, esm1b, temp=False):
    
    
    df_clean = removeoutlier_col(df,'Log'+output).copy()
    df_clean = df_clean.set_index('Index')
    

    X, y, X_scaled, scaler = encode(encoding, output, df_clean, aln, esm1b) 
    
    y_scaled = y

    X_scaled, y_scaled = shuffle(X_scaled, y_scaled, random_state=101)
    
    X, y = shuffle(X, y, random_state=101)
    

    lm = LinearRegression()   
    scores_lm = cross_validate(lm, X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_lm.update({"Algorithm": "Linear regression", 'Sequence Representation Method': encoding})
                               
    lasso_reg=Lasso()
    scores_lasso = cross_validate(lasso_reg,  X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_lasso.update({"Algorithm": "LASSO", 'Sequence Representation Method': encoding})
                               
    rf = RandomForestRegressor(random_state=42)
    scores_rf = cross_validate(rf,  X, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_rf.update({"Algorithm": "Random Forest", 'Sequence Representation Method': encoding})    
    
    tree_reg=DecisionTreeRegressor()
    scores_tree_reg = cross_validate(tree_reg,  X, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_tree_reg.update({"Algorithm": "Decision Tree", 'Sequence Representation Method': encoding})    
    
    svr_reg = SVR()
    scores_svr_reg = cross_validate(svr_reg,  X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_svr_reg.update({"Algorithm": "SVR", 'Sequence Representation Method': encoding})    
        
    
    mlp_reg = MLPRegressor(random_state=101, max_iter=100)
    scores_mlp_reg = cross_validate(mlp_reg,  X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_mlp_reg.update({"Algorithm": "Neural Network", 'Sequence Representation Method': encoding})    
    
    en = ElasticNet()
    scores_en = cross_validate(en, X_scaled, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_tree_reg.update({"Algorithm": "Elastic Network", 'Sequence Representation Method': encoding})    
    
    xgb_reg = xgb.XGBRegressor()
    scores_xgb_reg = cross_validate(xgb_reg,  X, y, cv=5,scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'), return_train_score=True)
    scores_xgb_reg.update({"Algorithm": "XGBoost", 'Sequence Representation Method': encoding})    
    

   

    dfResults = pd.concat([pd.DataFrame(scores_lm), pd.DataFrame(scores_lasso), pd.DataFrame(scores_rf), pd.DataFrame(scores_tree_reg), pd.DataFrame(scores_svr_reg), pd.DataFrame(scores_mlp_reg), pd.DataFrame(scores_tree_reg), pd.DataFrame(scores_xgb_reg)])
        
    
    return dfResults


In [6]:
enzyme = 'betaGlucosidasewithMutants'

df = pd.read_excel('betaGlucosidasewithMutantsOptimumTemperature.xlsx')

output = 'pNP-Glc kcat/Km (1/smM)'

methods = ['One-Hot-Encoder', 'Bag-of-Words', 'bigram', 'trigram', 'quadrogram', 'BLOSUM45', 'BLOSUM62', 'ESM1b']

aln = enzyme +'.aln'

x = datetime.datetime.now()
date = str(x.year)+str(x.month)+str(x.day)

df['y_val']= df['Temperature Optimum']
df['Log'+output]= np.log10(df[output])




In [7]:
esm1b = pd.read_excel(enzyme+'ESM1b_embeddings.xlsx', index_col = 0)

In [8]:
df_kcatKmTopt = df[df['Percentage Activity Depending on Optimum Temp']==1]

In [9]:
summary=[]

for method in methods:
    dfR = ml_process(method, output , df_kcatKmTopt, aln, esm1b, temp = False)
    print(dfR)
    summary.append(dfR)



C:\Users\memre\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.311e+01, tolerance: 1.561e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.837e+01, tolerance: 1.611e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.882e+02, toleranc

    fit_time  score_time       test_r2  train_r2  \
0   0.148213    0.005004 -1.269087e+15  0.999985   
1   0.173395    0.005008 -3.016708e+16  0.999981   
2   0.145682    0.005225  8.398187e-01  1.000000   
3   0.145857    0.004519  8.394515e-01  1.000000   
4   0.129544    0.004201 -2.100350e+18  0.999982   
0   0.969671    0.006446  8.647216e-01  0.971121   
1   1.077972    0.005550  7.396476e-01  0.975284   
2   1.110911    0.005037  8.688885e-01  0.969348   
3   1.186677    0.004004  8.493642e-01  0.972944   
4   1.153245    0.004516  9.000625e-01  0.973217   
0  25.456560    0.015511  8.855356e-01  0.978858   
1  23.309983    0.017011  7.548174e-01  0.984920   
2  25.837653    0.016331  8.716707e-01  0.978549   
3  25.560710    0.014006  8.378595e-01  0.976872   
4  25.556198    0.009399  9.062019e-01  0.978027   
0   0.497184    0.004003  7.857084e-01  1.000000   
1   0.389964    0.004224  5.580917e-01  1.000000   
2   0.500880    0.004004  7.748061e-01  1.000000   
3   0.477849

C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multi

   fit_time  score_time   test_r2  train_r2  test_neg_root_mean_squared_error  \
0  0.001229    0.001521  0.785126  0.831878                        -13.034762   
1  0.002000    0.001000  0.751797  0.829945                        -13.208104   
2  0.001136    0.001531  0.691705  0.848683                        -15.372818   
3  0.002008    0.001000  0.592654  0.838266                        -18.102547   
4  0.000211    0.001531  0.718285  0.833904                        -13.284071   
0  0.001000    0.002000  0.748224  0.772005                        -14.109708   
1  0.001154    0.001530  0.712152  0.770134                        -14.223914   
2  0.001001    0.002001  0.710010  0.789829                        -14.909475   
3  0.001702    0.002006  0.626941  0.771209                        -17.323944   
4  0.002000    0.002000  0.702685  0.772771                        -13.646916   
0  0.462112    0.011789  0.871046  0.981206                        -10.097849   
1  0.400046    0.010923  0.8

C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multi

...

    fit_time  score_time     test_r2  train_r2  \
0   0.016668    0.003690  -43.561864  0.999985   
1   0.007679    0.001517 -239.173040  0.999981   
2   0.010694    0.002999  -45.756963  1.000000   
3   0.010700    0.003006  -81.063410  1.000000   
4   0.008935    0.003073 -376.424457  0.999982   
0   0.028097    0.003008    0.840855  0.838534   
1   0.040814    0.003215    0.786557  0.861197   
2   0.039410    0.001997    0.750829  0.844937   
3   0.048183    0.003687    0.757203  0.830191   
4   0.040454    0.001995    0.844888  0.842782   
0   8.414856    0.009822    0.884231  0.978414   
1   7.430734    0.011763    0.768752  0.982324   
2   7.355283    0.008543    0.879162  0.978189   
3  10.695660    0.017568    0.826342  0.977058   
4   9.935907    0.022396    0.850153  0.976304   
0   0.159788    0.003339    0.633995  1.000000   
1   0.155067    0.003491    0.669978  1.000000   
2   0.116379    0.003019    0.815395  1.000000   
3   0.131928    0.002013    0.857684  1.000000   


C:\Users\memre\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.746e+01, tolerance: 1.561e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.797e+01, tolerance: 1.611e+01
  model = cd_fast.enet_coordinate_descent(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.607e+01, toleranc

    fit_time  score_time     test_r2  train_r2  \
0   0.012751    0.002537  -37.303434  0.999985   
1   0.008542    0.002705 -152.830118  0.999981   
2   0.010721    0.003000  -63.656297  1.000000   
3   0.010281    0.003520 -222.567550  1.000000   
4   0.011858    0.003516 -375.656448  0.999982   
0   0.043717    0.003000    0.836898  0.836371   
1   0.046891    0.002056    0.785462  0.858854   
2   0.052397    0.002997    0.749465  0.843325   
3   0.048112    0.003240    0.751645  0.828314   
4   0.047926    0.004002    0.841440  0.839607   
0  10.814011    0.014998    0.870906  0.980727   
1   8.479193    0.009774    0.773577  0.981942   
2   8.812259    0.013372    0.880043  0.976373   
3   8.888094    0.010360    0.832831  0.978977   
4   8.852528    0.012930    0.850588  0.978432   
0   0.127477    0.002001    0.770509  1.000000   
1   0.086553    0.003002    0.585607  1.000000   
2   0.091387    0.003006    0.801020  1.000000   
3   0.108360    0.001995    0.823207  1.000000   


C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\memre\anaconda3\lib\site-packages\sklearn\neural_network\_multi

    fit_time  score_time   test_r2  train_r2  \
0   0.029880    0.003507  0.509915  1.000000   
1   0.022005    0.003000  0.507448  1.000000   
2   0.021594    0.003012  0.083561  1.000000   
3   0.022897    0.004004 -0.198673  1.000000   
4   0.023216    0.002999 -0.645508  1.000000   
0   0.062406    0.003000  0.883292  0.911950   
1   0.076813    0.002000  0.853662  0.919928   
2   0.060511    0.001998  0.853307  0.917462   
3   0.075875    0.003000  0.855033  0.918782   
4   0.049929    0.002012  0.834478  0.909860   
0  44.128869    0.017402  0.903529  0.983782   
1  42.851969    0.018085  0.851411  0.984813   
2  48.703778    0.012122  0.883557  0.982791   
3  45.604509    0.012861  0.893718  0.983596   
4  43.906337    0.008528  0.892061  0.983200   
0   0.598464    0.004002  0.798332  1.000000   
1   0.553771    0.002227  0.781999  1.000000   
2   0.633530    0.004794  0.793439  1.000000   
3   0.531534    0.002601  0.800340  1.000000   
4   0.620675    0.005445  0.819303  1.00

In [10]:
result=pd.DataFrame()
for item in range(8):
    result=result.append(summary[item])
result.to_excel(date + 'Predicting Optimum Temperature' + enzyme +'5 CV.xlsx')

C:\Users\memre\AppData\Local\Temp\ipykernel_22720\1172027454.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_22720\1172027454.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_22720\1172027454.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_22720\1172027454.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(summary[item])
C:\Users\memre\AppData\Local\Temp\ipykernel_2272

In [11]:
df_res=result

In [12]:
df_res['test_root_mean_squared_error']=df_res['test_neg_root_mean_squared_error'].abs()
df_res['test_mean_absolute_error']=df_res['test_neg_mean_absolute_error'].abs()

df_res['train_root_mean_squared_error']=df_res['train_neg_root_mean_squared_error'].abs()
df_res['train_mean_absolute_error']=df_res['train_neg_mean_absolute_error'].abs()

In [13]:
test_res=df_res.groupby(['Algorithm', 'Sequence Representation Method'], as_index=False).agg({'test_r2':['mean','std'],
                                                                                             'test_root_mean_squared_error':['mean','std'],
                                                                                             'test_mean_absolute_error':['mean','std'],
                                                                                             'train_r2':['mean','std'],
                                                                                             'train_root_mean_squared_error':['mean','std'],
                                                                                             'train_mean_absolute_error':['mean','std']})

In [14]:
test_res.to_excel(date + 'Predicting Optimum Temperature' + enzyme +'5 CV mean and std.xlsx')